# Stop Loss VS Trail Stop VS ATR related SL && TrailSt

In [57]:
import pandas as pd
df = pd.read_csv("EURUSD_Candlestick_1_D_ASK_05.05.2003-30.06.2021.csv")
df.columns = ['Local time', 'open', 'high', 'low', 'close', 'volume']

#Check if NA values are in data
df=df[df['volume']!=0]
df.isna().sum()
df.reset_index(drop=True, inplace=True)

import pandas_ta as pa
df['ATR'] = pa.atr(high=df.high, low=df.low, close=df.close, length=14)
df.tail(20)

,Local time,open,high,low,close,volume,ATR
4714,03.06.2021 00:00:00.000 GMT+0300,1.22110,1.22145,1.21185,1.21286,88254.06500,0.006578
4715,04.06.2021 00:00:00.000 GMT+0300,1.21286,1.21859,1.21042,1.21692,93227.20000,0.006692
4716,07.06.2021 00:00:00.000 GMT+0300,1.21653,1.22020,1.21451,1.21900,54728.21500,0.006621
4717,08.06.2021 00:00:00.000 GMT+0300,1.21900,1.21944,1.21645,1.21738,89944.55750,0.006361
4718,09.06.2021 00:00:00.000 GMT+0300,1.21735,1.22182,1.21715,1.21827,58199.15500,0.006240
4719,10.06.2021 00:00:00.000 GMT+0300,1.21827,1.21949,1.21436,1.21759,90672.80000,0.006161
4720,11.06.2021 00:00:00.000 GMT+0300,1.21763,1.21932,1.20928,1.21096,66907.59500,0.006438
4721,14.06.2021 00:00:00.000 GMT+0300,1.21105,1.21307,1.20944,1.21211,52083.78350,0.006238
4722,15.06.2021 00:00:00.000 GMT+0300,1.21215,1.21475,1.21014,1.21273,65038.75000,0.006121
4723,16.06.2021 00:00:00.000 GMT+0300,1.21270,1.21350,1.19946,1.19952,100176.68500,0.006687


# Support and Resistance FUNCTIONS

In [58]:
def support(df1, l, n1, n2): #n1 n2 before and after candle l
    for i in range(l-n1+1, l+1):
        if(df1.low[i]>df1.low[i-1]):
            return 0
    for i in range(l+1,l+n2+1):
        if(df1.low[i]<df1.low[i-1]):
            return 0
    return 1

def resistance(df1, l, n1, n2): #n1 n2 before and after candle l
    for i in range(l-n1+1, l+1):
        if(df1.high[i]<df1.high[i-1]):
            return 0
    for i in range(l+1,l+n2+1):
        if(df1.high[i]>df1.high[i-1]):
            return 0
    return 1

In [59]:
length = len(df)
high = list(df['high'])
low = list(df['low'])
close = list(df['close'])
open = list(df['open'])
bodydiff = [0] * length

highdiff = [0] * length
lowdiff = [0] * length
ratio1 = [0] * length
ratio2 = [0] * length

# EurUSD set
mybodydiff = 0.000001
mybodydiffmin = 0.002

def isEngulfing(l):
    row=l
    bodydiff[row] = abs(open[row]-close[row])
    if bodydiff[row]<mybodydiff:
        bodydiff[row]=mybodydiff      

    bodydiffmin = mybodydiffmin
    if (bodydiff[row]>bodydiffmin and bodydiff[row-1]>bodydiffmin and
        open[row-1]<close[row-1] and
        open[row]>close[row] and 
        (open[row]-close[row-1])>=-0e-5 and close[row]<open[row-1]): #+0e-5 -5e-5
        return 1

    elif(bodydiff[row]>bodydiffmin and bodydiff[row-1]>bodydiffmin and
        open[row-1]>close[row-1] and
        open[row]<close[row] and 
        (open[row]-close[row-1])<=+0e-5 and close[row]>open[row-1]):#-0e-5 +5e-5
        return 2
    else:
        return 0

def isStar(l):
    bodydiffmin = mybodydiffmin
    row=l
    highdiff[row] = high[row]-max(open[row],close[row])
    lowdiff[row] = min(open[row],close[row])-low[row]
    bodydiff[row] = abs(open[row]-close[row])
    if bodydiff[row]<mybodydiff:
        bodydiff[row]=mybodydiff
    ratio1[row] = highdiff[row]/bodydiff[row]
    ratio2[row] = lowdiff[row]/bodydiff[row]

    if (ratio1[row]>1 and lowdiff[row]<0.2*highdiff[row] and bodydiff[row]>bodydiffmin):# and open[row]>close[row]):
        return 1
    elif (ratio2[row]>1 and highdiff[row]<0.2*lowdiff[row] and bodydiff[row]>bodydiffmin):# and open[row]<close[row]):
        return 2
    else:
        return 0
    
def closeResistance(l,levels,lim):
    if len(levels)==0:
        return 0
    #!!!
    #lim=df.ATR[l]/2
    
    #diff between high and closest level among levels
    c1 = abs(df.high[l]-min(levels, key=lambda x:abs(x-df.high[l])))<=lim
    #diff between higher body and closest level to high
    c2 = abs(max(df.open[l],df.close[l])-min(levels, key=lambda x:abs(x-df.high[l])))<=lim
    #min body less than closest level to high
    c3 = min(df.open[l],df.close[l])<min(levels, key=lambda x:abs(x-df.high[l]))
    #low price less than closest level to high
    c4 = df.low[l]<min(levels, key=lambda x:abs(x-df.high[l]))
    if( (c1 or c2) and c3 and c4 ):
        return 1
    else:
        return 0
    
def closeSupport(l,levels,lim):
    if len(levels)==0:
        return 0
    #!!!
    #lim=df.ATR[l]/2
    c1 = abs(df.low[l]-min(levels, key=lambda x:abs(x-df.low[l])))<=lim
    c2 = abs(min(df.open[l],df.close[l])-min(levels, key=lambda x:abs(x-df.low[l])))<=lim
    c3 = max(df.open[l],df.close[l])>min(levels, key=lambda x:abs(x-df.low[l]))
    c4 = df.high[l]>min(levels, key=lambda x:abs(x-df.low[l]))
    if( (c1 or c2) and c3 and c4 ):
        return 1
    else:
        return 0

In [60]:
n1=2
n2=2
backCandles=45
signal = [0] * length

for row in range(backCandles, len(df)-n2):
    ss = []
    rr = []
    for subrow in range(row-backCandles+n1, row+1):
        if support(df, subrow, n1, n2):
            ss.append(df.low[subrow])
        if resistance(df, subrow, n1, n2):
            rr.append(df.high[subrow])
    
    #!!!! parameters
    myclosedistance = 150e-5 #EURUSD
    if ((isEngulfing(row)==1 or isStar(row)==1) and closeResistance(row, rr, myclosedistance) ):#and df.RSI[row]<30
        signal[row] = 1
    elif((isEngulfing(row)==2 or isStar(row)==2) and closeSupport(row, ss, myclosedistance)):#and df.RSI[row]>70
        signal[row] = 2
    else:
        signal[row] = 0



In [61]:
df['signal']=signal

In [62]:
print(df[df['signal']==1].count(),df[df['signal']==2].count())

Local time    98
open          98
high          98
low           98
close         98
volume        98
ATR           98
signal        98
dtype: int64 Local time    87
open          87
high          87
low           87
close         87
volume        87
ATR           87
signal        87
dtype: int64


In [63]:
df.columns = ['Local time', 'Open', 'High', 'Low', 'Close', 'Volume', 'ATR', 'signal']
#df=df.iloc[100:200]
#help(pa.atr)
df

,Local time,Open,High,Low,Close,Volume,ATR,signal
0,05.05.2003 00:00:00.000 GMT+0300,1.12354,1.13019,1.12030,1.12804,1.053163e+06,NaN,0
1,06.05.2003 00:00:00.000 GMT+0300,1.12798,1.14510,1.12738,1.14368,1.061123e+06,NaN,0
2,07.05.2003 00:00:00.000 GMT+0300,1.14367,1.14448,1.13275,1.13677,1.056797e+06,NaN,0
3,08.05.2003 00:00:00.000 GMT+0300,1.13675,1.15092,1.13026,1.15026,1.058909e+06,NaN,0
4,09.05.2003 00:00:00.000 GMT+0300,1.15002,1.15376,1.14362,1.14907,1.063989e+06,NaN,0
...,...,...,...,...,...,...,...,...
4729,24.06.2021 00:00:00.000 GMT+0300,1.19267,1.19565,1.19178,1.19322,8.515221e+04,0.006791,0
4730,25.06.2021 00:00:00.000 GMT+0300,1.19322,1.19754,1.19264,1.19392,7.783765e+04,0.006656,0
4731,28.06.2021 00:00:00.000 GMT+0300,1.19380,1.19447,1.19025,1.19260,8.515426e+04,0.006482,0
4732,29.06.2021 00:00:00.000 GMT+0300,1.19297,1.19334,1.18779,1.18973,9.889857e+04,0.006415,0


In [64]:
def SIGNAL():
    return df.signal

In [65]:
#A new strategy needs to extend Strategy class and override its two abstract methods: init() and next().
#Method init() is invoked before the strategy is run. Within it, one ideally precomputes in efficient, 
#vectorized manner whatever indicators and signals the strategy depends on.
#Method next() is then iteratively called by the Backtest instance, once for each data point (data frame row), 
#simulating the incremental availability of each new full candlestick bar.

#Note, backtesting.py cannot make decisions / trades within candlesticks — any new orders are executed on the
#next candle's open (or the current candle's close if trade_on_close=True). 
#If you find yourself wishing to trade within candlesticks (e.g. daytrading), you instead need to begin 
#with more fine-grained (e.g. hourly) data.

In [122]:
# Trader fixed SL and TP
from backtesting import Strategy, Backtest
class MyCandlesStrat(Strategy):  
    def init(self):
        super().init()
        self.signal1 = self.I(SIGNAL)

    def next(self):
        super().next() 
        if self.signal1==2:
            sl1 = self.data.Close[-1] - 550e-4 #EURUSD
            tp1 = self.data.Close[-1] + 600e-4
            self.buy(sl=sl1, tp=tp1)
        elif self.signal1==1:
            sl1 = self.data.Close[-1] + 550e-4 #EURUSD
            tp1 = self.data.Close[-1] - 600e-4
            self.sell(sl=sl1, tp=tp1)
bt = Backtest(df, MyCandlesStrat, cash=10_000, commission=.000)
stat = bt.run()
stat

D:\ProgramFiles\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: UserWarning: Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.


Start                                     0.0
End                                    4733.0
Duration                               4733.0
Exposure Time [%]                   75.369666
Equity Final [$]                  22107.96754
Equity Peak [$]                   22595.45781
Return [%]                         121.079675
Buy & Hold Return [%]                5.128364
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                  -21.539101
Avg. Drawdown [%]                   -2.072615
Max. Drawdown Duration                 1801.0
Avg. Drawdown Duration              60.295775
# Trades                                 98.0
Win Rate [%]                        70.408163
Best Trade [%]                       5.638716
Worst Trade [%]                     -5.052092
Avg. Trade [%]                    

In [110]:
# ATR related SL and TP
from backtesting import Strategy, Backtest
import numpy as np


class MyCandlesStrat(Strategy): 
    atr_f = 0.2
    ratio_f = 1
    def init(self):
        super().init()
        self.signal1 = self.I(SIGNAL)

    def next(self):
        super().next() 
        if self.signal1==2:
            sl1 = self.data.Close[-1] - self.data.ATR[-1]/self.atr_f
            tp1 = self.data.Close[-1] + self.data.ATR[-1]*self.ratio_f/self.atr_f
            self.buy(sl=sl1, tp=tp1)
        elif self.signal1==1:
            sl1 = self.data.Close[-1] + self.data.ATR[-1]/self.atr_f
            tp1 = self.data.Close[-1] - self.data.ATR[-1]*self.ratio_f/self.atr_f
            self.sell(sl=sl1, tp=tp1)
bt = Backtest(df, MyCandlesStrat, cash=10_000, commission=.000)
stat = bt.run()
stat

D:\ProgramFiles\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: UserWarning: Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.


Start                                     0.0
End                                    4733.0
Duration                               4733.0
Exposure Time [%]                   76.362484
Equity Final [$]                 22540.846509
Equity Peak [$]                  25841.713272
Return [%]                         125.408465
Buy & Hold Return [%]                5.128364
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                  -23.472182
Avg. Drawdown [%]                     -2.1381
Max. Drawdown Duration                 1626.0
Avg. Drawdown Duration              46.933333
# Trades                                101.0
Win Rate [%]                        71.287129
Best Trade [%]                      10.856063
Worst Trade [%]                      -7.72418
Avg. Trade [%]                    

In [114]:
#fixed distance Trailing SL
from backtesting import Strategy, Backtest
import numpy as np

class MyCandlesStrat(Strategy):
    sltr=500e-4 #EURUSD
    def init(self):
        super().init()
        self.signal1 = self.I(SIGNAL)

    def next(self):
        super().next()
        sltr = self.sltr
        for trade in self.trades: 
            if trade.is_long: 
                trade.sl = max(trade.sl or -np.inf, self.data.Close[-1] - sltr)
            else:
                trade.sl = min(trade.sl or np.inf, self.data.Close[-1] + sltr) 

        if self.signal1==2 and len(self.trades)==0: # trades number change!
            sl1 = self.data.Close[-1] - sltr
            self.buy(sl=sl1)
        elif self.signal1==1 and len(self.trades)==0: # trades number change!
            sl1 = self.data.Close[-1] + sltr
            self.sell(sl=sl1)


bt = Backtest(df, MyCandlesStrat, cash=10_000, commission=.000)
stat = bt.run()
stat

D:\ProgramFiles\Anaconda3\lib\site-packages\ipykernel_launcher.py:28: UserWarning: Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.


Start                                     0.0
End                                    4733.0
Duration                               4733.0
Exposure Time [%]                   75.327419
Equity Final [$]                  23178.80878
Equity Peak [$]                   23665.54081
Return [%]                         131.788088
Buy & Hold Return [%]                5.128364
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                  -16.444184
Avg. Drawdown [%]                   -1.677121
Max. Drawdown Duration                 1111.0
Avg. Drawdown Duration               41.53211
# Trades                                 40.0
Win Rate [%]                             55.0
Best Trade [%]                      20.533811
Worst Trade [%]                     -3.768513
Avg. Trade [%]                    

In [121]:
#ATR distance Trailing SL
from backtesting import Strategy, Backtest
import numpy as np

class MyCandlesStrat(Strategy):
    atr_f = 1
    def init(self):
        super().init()
        self.signal1 = self.I(SIGNAL)

    def next(self):
        super().next()
        for trade in self.trades: 
            if trade.is_long: 
                trade.sl = max(trade.sl or -np.inf, self.data.Close[-1] - self.data.ATR[-1]/self.atr_f)
            else:
                trade.sl = min(trade.sl or np.inf, self.data.Close[-1] + self.data.ATR[-1]/self.atr_f)

        if self.signal1==2 and len(self.trades)==0: # trades number change!
            sl1 = self.data.Close[-1] - self.data.ATR[-1]/self.atr_f
            self.buy(sl=sl1)
        elif self.signal1==1 and len(self.trades)==0: # trades number change!
            sl1 = self.data.Close[-1] + self.data.ATR[-1]/self.atr_f
            self.sell(sl=sl1)
bt = Backtest(df, MyCandlesStrat, cash=10_000, commission=.000)
stat = bt.run()
stat

D:\ProgramFiles\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: UserWarning: Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.


Start                                     0.0
End                                    4733.0
Duration                               4733.0
Exposure Time [%]                   20.616815
Equity Final [$]                 16967.842679
Equity Peak [$]                  17622.459802
Return [%]                          69.678427
Buy & Hold Return [%]                5.128364
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                   -7.112805
Avg. Drawdown [%]                   -1.297683
Max. Drawdown Duration                  783.0
Avg. Drawdown Duration              68.104478
# Trades                                159.0
Win Rate [%]                        48.427673
Best Trade [%]                       6.308715
Worst Trade [%]                     -1.910493
Avg. Trade [%]                    